In [8]:
import sys
import os

# Add the path to the aqi package
aqi_path = os.path.abspath('../../../airflow')
sys.path.append(aqi_path)

from plugins.hooks.station import fetch_data
from dags.station.etl import validate_data, parse_data, clean_data, push_to_db

In [9]:
aqi_data = fetch_data()
print(aqi_data['status'])
print(len(aqi_data['data']))

<Response [200]>
{'status': 'ok', 'data': [{'lat': 13.917563, 'lon': 101.590529, 'uid': 9014, 'aqi': '58', 'station': {'name': 'Tha Tum, Si Maha Phot, Prachin Buri, Thailand (ต.ท่าตูม อ.ศรีมหาโพธิ จ.ปราจีนบุรี)', 'time': '2025-02-24T23:00:00+09:00'}}, {'lat': 13.852946, 'lon': 100.561281, 'uid': -421522, 'aqi': '76', 'station': {'name': 'บริษัท ดีดีมาร์ท โฮลดิ้ง จำกัด', 'time': '2025-02-24T23:18:26+09:00'}}, {'lat': 14.022376, 'lon': 99.536195, 'uid': 10481, 'aqi': '84', 'station': {'name': 'Pak Prak, Kanchanaburi, Kanchanaburi, Thailand (ต.ปากแพรก อ.เมือง จ.กาญจนบุรี)', 'time': '2025-02-24T22:00:00+09:00'}}, {'lat': 18.739273, 'lon': 99.079945, 'uid': -458929, 'aqi': '69', 'station': {'name': 'บ้านแจ่มใสคลินิกเวชกรรม 2', 'time': '2025-02-24T23:18:25+09:00'}}, {'lat': 18.840732, 'lon': 98.96978, 'uid': 6817, 'aqi': '76', 'station': {'name': 'City Hall, Chiangmai, Thailand (ศาลากลางเชียงใหม่)', 'time': '2025-02-24T22:00:00+09:00'}}, {'lat': 7.982461, 'lon': 98.362088, 'uid': -419716, 'a

In [10]:
validate_data(aqi_data)

In [11]:
parse_aqi_data = parse_data(aqi_data)
print(parse_aqi_data[:3])

      uid aqi   latitude   longitude  \
0    9014  58  13.917563  101.590529   
1 -421522  76  13.852946  100.561281   
2   10481  84  14.022376   99.536195   

                                        station_name                 timestamp  
0  Tha Tum, Si Maha Phot, Prachin Buri, Thailand ... 2025-02-24 23:00:00+09:00  
1                     บริษัท ดีดีมาร์ท โฮลดิ้ง จำกัด 2025-02-24 23:18:26+09:00  
2  Pak Prak, Kanchanaburi, Kanchanaburi, Thailand... 2025-02-24 22:00:00+09:00  


In [12]:
clean_aqi_data = clean_data(parse_aqi_data)
print(clean_aqi_data[:3])

      uid   aqi   latitude   longitude  \
0    9014  58.0  13.917563  101.590529   
1 -421522  76.0  13.852946  100.561281   
2   10481  84.0  14.022376   99.536195   

                                        station_name                 timestamp  
0  Tha Tum, Si Maha Phot, Prachin Buri, Thailand ... 2025-02-24 23:00:00+09:00  
1                     บริษัท ดีดีมาร์ท โฮลดิ้ง จำกัด 2025-02-24 23:18:26+09:00  
2  Pak Prak, Kanchanaburi, Kanchanaburi, Thailand... 2025-02-24 22:00:00+09:00  


In [10]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Push the cleaned AQI data to the database
push_data = push_to_db(conn, clean_aqi_data)

# Close the connection
conn.close()